<a href="https://colab.research.google.com/github/alibakh62/DeepFake-Kaggle/blob/master/notebook/colab/DeepFakeDetection_training_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configs and Imports

In [1]:
!pip install ipython-autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=ef5a4ddca5926859ea8ed1de17f0057d756b4ee723eb486e7a56dd358d04e084
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [2]:
%load_ext autotime
%tensorflow_version 2.x
%matplotlib inline
%reload_ext tensorboard

import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [3]:
# verify TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

Running on TPU  ['10.17.40.18:8470']
time: 2.18 ms


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import cv2 as cv
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils.validation import column_or_1d
from sklearn.metrics import accuracy_score

time: 623 ms


In [5]:
# configs
MODELS_FOLDER = '/content/drive/My Drive/DeepFake/models'
BASE_FOLDER = '/content/drive/My Drive/DeepFake'
TRAIN_SAMPLE_VIDEOS = 'train_sample_videos'
TRAIN_INPUT = 'input'
TEST_VIDEOS = 'test_videos'
TRAIN_FRAMES_FOLDER = 'train_frames'
VALID_FRAMES_FOLDER = 'valid_frames'
TEST_FRAMES_FOLDER = 'test_frames'
DATA_FOLDER = 'data'
TRAIN_FOLDER = 'train'
VALID_FOLDER = 'valid'
TEST_FOLDER = 'test'
TRAIN_FACE = 'train_face'
VALID_FACE = 'valid_face'
METADATA = 'metadata'
INPUT_SHAPE = (299, 299)  # usually the input size for pretrained models  
BATCH_SIZE = 128

time: 4.98 ms


# Training Data

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(rescale=1./255)
# valid_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.15,
                                   horizontal_flip=True,
                                   fill_mode="nearest")
valid_datagen = ImageDataGenerator(rescale=1./255)

train_dir = os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FACE)
valid_dir = os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FACE)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=INPUT_SHAPE,
    batch_size=BATCH_SIZE,
    classes=['REAL', 'FAKE'],
    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=INPUT_SHAPE,
    batch_size=BATCH_SIZE,
    classes=['REAL', 'FAKE'],
    class_mode='binary')

Found 15853 images belonging to 2 classes.
Found 4186 images belonging to 2 classes.
time: 3.58 s


In [12]:
valid_generator.class_indices

{'FAKE': 1, 'REAL': 0}

time: 2.51 ms


In [13]:
print(f"Total REAL train: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FACE, 'REAL')))}")
print(f"Total FAKE train: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, TRAIN_FACE, 'FAKE')))}")
print(f"Total REAL valid: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FACE, 'REAL')))}")
print(f"Total FAKE valid: {len(os.listdir(os.path.join(BASE_FOLDER, DATA_FOLDER, VALID_FACE, 'FAKE')))}")

Total REAL train: 1626
Total FAKE train: 14227
Total REAL valid: 552
Total FAKE valid: 3634
time: 122 ms


# Models

### Base model: Xception

In [15]:
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions

base_model = Xception(include_top=False, weights="imagenet")
base_model.trainable = False

83689472/83683744 [==============================] - 1s 0us/step
time: 3.21 s


### Base model: VGG

### Base model: InceptionV3

### Attention model

In [20]:
pt_features = Input(base_model.get_output_shape_at(0)[1:], name='feature_input')
pt_depth = base_model.get_output_shape_at(0)[-1]

time: 3.25 ms


In [21]:
bn_features = BatchNormalization()(pt_features)

time: 37.4 ms


In [22]:
attn_layer = Conv2D(128, kernel_size=(1,1), padding='same', activation = 'elu')(bn_features)
attn_layer = Conv2D(32, kernel_size=(1,1), padding='same', activation = 'elu')(attn_layer)
attn_layer = Conv2D(16, kernel_size=(1,1), padding='same', activation = 'elu')(attn_layer)
attn_layer = AvgPool2D((2,2), strides=(1,1), padding='same')(attn_layer)
attn_layer = Conv2D(1, kernel_size=(1,1), padding='valid', activation = 'sigmoid')(attn_layer)

time: 45.9 ms


In [23]:
up_c2_w = np.ones((1, 1, 1, pt_depth))
up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
               activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

time: 10.4 ms


In [24]:
mask_features = multiply([attn_layer, bn_features])

time: 6.37 ms


In [25]:
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.5)(gap)
gap_features1 = GlobalAveragePooling2D()(mask_features)
gap_mask1 = GlobalAveragePooling2D()(attn_layer)
#to account for missing values from the attention model
gap1 = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features1, gap_mask1])
gap_dr1 = Dropout(0.5)(gap1)
gap_features2 = GlobalAveragePooling2D()(mask_features)
gap_mask2 = GlobalAveragePooling2D()(attn_layer)
gap2 = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features2, gap_mask2])
gap_dr2 = Dropout(0.5)(gap2)

time: 79.2 ms


In [26]:
dr_steps = Dropout(0.5)(Dense(128, activation = 'elu')(gap_dr2))
out_layer = Dense(1, activation = 'sigmoid')(dr_steps)

attn_model = Model(inputs = [pt_features], outputs = [out_layer], name = 'attention_model')

attn_model.compile(optimizer = Adam(lr = 1e-3, beta_1=0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = True),
              loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

attn_model.summary()

Model: "attention_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
feature_input (InputLayer)      [(None, None, None,  0                                            
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, None, None, 2 8192        feature_input[0][0]              
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, None, None, 1 262272      batch_normalization_4[0][0]      
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, None, None, 3 4128        conv2d_4[0][0]                   
____________________________________________________________________________________

### Model compile and summary

In [27]:
model = Sequential(name='combined_model')
model.add(base_model)
model.add(attn_model)
model.compile(optimizer = Adam(lr = 1e-3, beta_1=0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = True),
              loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
model.summary()

Model: "combined_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
attention_model (Model)      (None, 1)                 539586    
Total params: 21,401,066
Trainable params: 533,442
Non-trainable params: 20,867,624
_________________________________________________________________
time: 1.38 s


### Checkpoints and callbacks

In [29]:
# checkpoint callback
from datetime import datetime
timestamp = datetime.now().strftime(format="%Y%m%d_%H%M%S")
CHECKPOINT = f"xception_attention_face_v1_augdata_17K_frames_{timestamp}.h5"
weights_path = os.path.join(MODELS_FOLDER, CHECKPOINT)

from tensorflow.keras.callbacks import ModelCheckpoint
ckp_loss = ModelCheckpoint(weights_path, save_best_only=True, verbose=1, 
                      monitor='val_loss', mode='min')
ckp_accu = ModelCheckpoint(weights_path, save_best_only=True, verbose=1, 
                      monitor='val_binary_accuracy', mode='max')

time: 3.35 ms


In [30]:
# early stopping callback
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_cb = EarlyStopping(patience=3, restore_best_weights=True)

time: 1.1 ms


In [31]:
# tensorboard callback
from tensorflow.keras.callbacks import TensorBoard

def get_run_logdir(rootlogdir):
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(rootlogdir, run_id)

root_logdir = os.path.join(BASE_FOLDER, 'logs')
run_logdir = get_run_logdir(root_logdir)
tensorboard_cb = TensorBoard(run_logdir)
print(run_logdir)

/content/drive/My Drive/DeepFake/logs/run_2020_03_30-17_41_58
time: 82.1 ms


### Training

In [0]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//BATCH_SIZE,
    epochs=50,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples//BATCH_SIZE,
    callbacks=[ckp_loss, ckp_accu, tensorboard_cb, early_stopping_cb])
model.save(weights_path)

Epoch 1/50
122/123 [============================>.] - ETA: 46s - loss: 0.3721 - binary_accuracy: 0.8801 